# Spectrum fitting tools
# スペクトルフィッティングツール
車輪の再発明的にフィッティングを一から作って、最終的にはスペクトルを分離フィッティングする。

## 参考にしたサイト
[Pythonで非線形関数モデリング](https://qiita.com/hik0107/items/9bdc236600635a0e61e8)  
[研究者のための実践データ処理～Pythonでピークフィッティング～](https://qiita.com/kon2/items/6498e66af55949b41a99)

In [1]:
# import files
import numpy as np
import matplotlib.pyplot as plt
from numpy import exp, log, inf, sum
from scipy.optimize import curve_fit

In [2]:
# Constant
LOOP_MAX = 100 # ループ回数の最大値

## ガウス関数
スペクトルのピークはガウス関数で近似できいると仮定する  
ガウス関数は  
$$
y = a\exp \biggl\{ -\frac{(x-b)^{2}}{2c^{2}} \biggr\}
$$
で表される。

In [3]:
def gaussian(x,a=1,b=0,c=1):
    """
    ガウス関数を返す
    Parameters
    ----------
    x : numpy.ndarray
    a : float
    b : float
    c : float
    Returns
    -------
    gaussian : numpy.ndarray
    """
    return a*exp(-(x-b)**2/(2*c**2))

## 単一のガウス関数フィッティング
### 初期値の設定  

- a: データ列の最大値  
- b: 最大値を与える位置
- c: 半値幅　　

を設定して計算させる
### 解析範囲の設定
Gauss関数の場合、半値幅（FWHM)は
$$
FWHM = 2\sqrt{2\ln{2}}\sigma \approx 2.35\sigma
$$
なので、FWHMを解析範囲に設定

In [4]:
def peak_info(data):
    """
    データの最大値の高さ、位置、半値幅を返す
    Parameters
    ----------
    data : numpy.ndarray
        入力データ
    Returns
    -------
    height : float
        データの最大値
    pos : int
        最大値を与えるインデックス
    width : int
        半値幅
    """
   
    def find_first_index(x,flag):
        """
        Boolean列でFlag値が最初に現れる位置を返す
        Parameters
        ----------
        x : list
            True/False
        flag : bool 
           True or False
        Returns
        -------
        find_first_index : int
            最初にflagとなったインデックス
        """
        for i in range(len(x)):
            if x[i]  == flag:
                break
        return i
    height = data.max()
    pos = data.argmax()
    low = find_first_index(data[pos:0:-1] < height/2,True)
    high = find_first_index(data[pos::] < height/2,True)
    width = low+high
    return height, pos, width

In [5]:
def fit_gaussian(data):
    """
    最大値付近をガウス関数でフィッティングする
    Parameters
    ----------
    data : numpy.ndarray
        入力データ
    Returns
    -------
    param : numpy.ndarray
        フィッティングしたピーク高さ
        フィッティングしたピーク位置
        フィッティングしたピークの半値幅
    """   
    p0 = peak_info(data)
    xs = np.arange(len(data)) # X列を生成
    bounds =((0,p0[1]-p0[2]/2,0),(inf,p0[1]+p0[2]/2,inf)) # 解析範囲を限定
    param ,_ =  curve_fit(gaussian, xs, data, p0=p0,bounds=bounds)
    return param

## ガウス関数をプロットする

In [6]:
def plot_gaussians(xs, params):
    """
    ガウス関数をプロットする
    Parameters
    ----------
    xs : numpy.ndarray
    params : list
        ガウスパラメータのリスト
    """
    for param in params:
        plt.plot(xs, gaussian(xs,a = param[0], b = param[1], c = param[2]))

## 多重ガウス関数

In [7]:
def gaussians(x,*params):    
    """
    多重版ガウス関数
    Parameters
    ----------
    x : numpy.ndarray
    *params : list
        ガウスパラメータのリスト
    Returns
    -------
    gaussians : numpy.ndarray
    """

    n = len(params)//3
    func = np.zeros_like(x,dtype=float)
    for i in range(n):
        func += gaussian(x,a=params[i*3],b=params[i*3+1],c=params[i*3+2])
    return func

### 平均二乗誤差(MSE)
実データ$x_{i}$とモデル$y_{i}$との間の誤差
$$
MSE = \frac{1}{n}\sum^{n}_{i=1}(x_{i}-y_{i})^2
$$
これの平方根を取ると
$$
RMSE = \sqrt{\frac{1}{n}\sum^{n}_{i=1}(x_{i}-y_{i})^2}
$$

In [8]:
def mse(xs,ys):
    """
    実データxsとモデルysの誤差MSEを計算する
    Parameters
    ----------
    xs : numpy.ndarray
        実データ
    ys : numpy.ndarray
        モデルデータ
    Returns
    -------
    mse : float
        平均二乗誤差
    """
    return sum((xs-ys)**2)/len(xs)

## 複数のガウス関数のフィッティフィッティング
1. データの最大値をピークとみなして、フィッティングする
1. 元のデータからフィッティングしたガウス関数を引いて、新しいデータとする
1. 残渣と残渣の平均値のMSEを計算
1. MSEが前回よりも大きい場合、ループを抜ける
1. 全てのガウス関数を足して合成関数を作成する
1. 合成関数でフィッティングを行う
1. 各ピークについて、覗いた場合のMSEを計算して、利得が最大かつ0dB以上のものを残す

In [9]:
def fit_gaussians(data, level = 3.0):
    """
    ｎ個のガウス関数でフィッティングする
    Parameters
    ----------
    data : numpy.array
        解析するデータ
    level : float
        感度
    Returns
    -------
    params : numpy.array
        ガウスパラメータ
    """

    xs = np.arange(len(data))
    residuals = data.copy()
    
    param_list = []
    l_limit = []
    h_limit = []
    
    prior_mse = mse(residuals, np.ones_like(residuals)*residuals.mean())
    for _ in range(LOOP_MAX):
        param = fit_gaussian(residuals)
        residuals -= gaussian(xs, a = param[0], b = param[1], c = param[2]) 
        post_mse = mse(residuals, np.ones_like(residuals)*residuals.mean())
        mse_db = -10*log(post_mse/prior_mse)
        if mse_db  < level:
            break
        param_list.extend(param)
        l_limit.extend([0,param[1]-param[2]/2,0])
        h_limit.extend([inf,param[1]+param[2]/2,inf])
        prior_mse = post_mse
    
    bounds = (l_limit,h_limit)
    param_list, _ = curve_fit(gaussians, xs, data, p0=param_list, bounds = bounds)
    
    n = len(param_list)//3
    print('Peak candidate:',n)

    while(n>1):
        l_limits=[]
        h_limits=[]
        dbs=[]
        param_lists = []
        for i in range(n):
            l_limits.extend([0,param_list[i*3+1]-param_list[i*3+2]/2,0])
            h_limits.extend([inf,param_list[i*3+1]+param_list[i*3+2]/2,inf])
        prior_mse = mse(data, gaussians(xs,*param_list))
        for i in range(n):
            new_param_list = np.concatenate([param_list[:i*3],param_list[i*3+3:]])
            new_l_limits = l_limits[:i*3]+l_limits[i*3+3:]
            new_h_limits = h_limits[:i*3]+h_limits[i*3+3:]
            bounds = (new_l_limits, new_h_limits)
            new_param_list, _ = curve_fit(gaussians,xs, data, 
                                          p0 = new_param_list,bounds=bounds)
            post_mse = mse(data, gaussians(xs,*new_param_list))
            param_lists.append(new_param_list)
            dbs.append(-10*log(post_mse/prior_mse))
        db_max=max(dbs)
        if db_max < 0:
            break
        param_list = param_lists[dbs.index(db_max)]
        n = len(param_list)//3
    params = param_list.reshape((len(param_list)//3),3)
    
    return params       

Peak candidate: 5
[[ 2.02 30.    4.  ]
 [ 2.   50.    2.  ]
 [ 2.   20.    5.  ]
 [ 2.   40.    3.  ]]


In [10]:
# TEST Code
xs = np.arange(100)
ys = gaussian(xs,a=2,b=20,c=5) + gaussian(xs,a=2.02,b=30,c=4) + \
     gaussian(xs,a=2,b=40,c=3) + gaussian(xs,a=2,b=50,c=2)
#ys = gaussian(xs,a=2,b=20,c=2)+gaussian(xs,a=2.2,b=25,c=2)
params = fit_gaussians(ys,level=.2)
plt.plot(xs,ys)
plot_gaussians(xs, params)
print(params)